In [1]:
import pandas as pd
import numpy as np
import re

In [4]:
df = pd.read_csv("./dataset/HTTPParamsDataset/payload_full.csv")
df.sample(10, random_state=44)

,payload,length,attack_type,label
11381,"1%"")) order by 1--",18,sqli,anom
5672,6411,4,norm,norm
6078,devette,7,norm,norm
4561,rufino@citycvs.as,17,norm,norm
8952,catll,5,norm,norm


In [11]:
attackTypes = set(df.attack_type)
labels = set(df.label)
d_attackTypes = { k : v for k,v in zip(attackTypes, range(len(attackTypes))) }
d_labels = { k : v for k,v in zip(labels, range(len(labels))) }

In [12]:
print(d_attackTypes)
print(d_labels)

{'norm': 0, 'path-traversal': 1, 'cmdi': 2, 'xss': 3, 'sqli': 4}
{'anom': 0, 'norm': 1}


In [13]:
df['attack_type_num'] = df.attack_type.apply(lambda x : d_attackTypes[x])
df['label_num'] = df.label.apply(lambda x : d_labels[x])

In [31]:
X = df['length']
y = df['label_num']

In [32]:
from input_clearing import *
# num_digits, num_spec_chars, num_nalpha_chars, num_payload_args
df1 = df.payload.apply(lambda row : pd.Series(gen_payload_info(row), index=['digits', 'spec_chars', 'not_alpha_chars', 'payload_args']))
X = pd.concat([X, df1], axis=1)

In [33]:
X.sample(10, random_state=44)

,length,digits,spec_chars,not_alpha_chars,payload_args
11381,18,2,3,11,0
5672,4,4,0,4,0
6078,7,0,0,0,0
4561,17,0,1,2,0
8952,5,0,0,0,0
7590,20,2,0,6,0
24980,26,0,1,2,0
19080,8,0,0,0,0
732,30,0,1,3,0
27019,47,16,7,34,1


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y)
clf=RandomForestClassifier(n_estimators=100)

In [35]:
clf.fit(X_train.values,y_train.values)
y_pred=clf.predict(X_test.values)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9951075061156174


In [36]:
clf.feature_importances_

array([0.14247087, 0.0313671 , 0.28056045, 0.50028091, 0.04532066])

In [37]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2963
           1       1.00      1.00      1.00      4804

    accuracy                           1.00      7767
   macro avg       0.99      0.99      0.99      7767
weighted avg       1.00      1.00      1.00      7767



In [38]:
import pickle
pickle.dump(clf, open('testmodel.pkl', 'wb'))